In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.optimize import minimize
%matplotlib inline

In [19]:
data = sio.loadmat('data/ex3data1.mat')
weights = sio.loadmat('data/ex3weights.mat')

In [20]:
data.keys()

dict_keys(['__version__', 'X', '__globals__', 'y', '__header__'])

In [21]:
weights.keys()

dict_keys(['__version__', 'Theta2', '__globals__', 'Theta1', '__header__'])

In [22]:
X = data['X']
y = data['y'].flatten()
theta1 = weights['Theta1']
theta2 = weights['Theta2']
para = np.append(theta1.flatten(), theta2.flatten())

In [23]:
temp = np.linspace(0, 9, 10)

# Feedforward and cost function

#### Cost function for the neural network

\begin{equation}
J(\theta) = \frac{1}{m}\sum_{i=1}^m \sum_{k=1}^K [-y_k^{(i)}\log((h_{\theta}(x^{(i)}))_k) -(1 - y_k^{(i)})\log(1-(h_{\theta}(x^{(i)}))_k) ]
\end{equation}

In [24]:
#sigmoid function
def sigm(x):
    return(1/(1+np.exp(-x)))

In [25]:
def sigmoidGradient(x):
    return(sigm(x)*(1-sigm(x)))

In [26]:
def randInitializeWeights(length, epsilon_init = 0.12):
    return(np.random.uniform(-epsilon_init, epsilon_init, length))    

In [27]:
def predict(Theta1, Theta2, X, m):
    a1 = np.column_stack((np.ones(m), X)) #5000x401
    z2 = a1.dot(Theta1.T) # 5000x401 * 401x25 = 5000x25 
    a2 = np.column_stack((np.ones(z2.shape[0]), sigm(z2))) # 5000x26
    z3 = a2.dot(Theta2.T) # 5000x26 * 26x10 = 5000x10
    a3 = sigm(z3) #5000x10
    return(a1, a2, a3, z2, z3)

In [28]:
#cost function
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lam):
    Theta1 = np.reshape(nn_params[:((input_layer_size+1)*hidden_layer_size)], (hidden_layer_size, input_layer_size+1))
    Theta2 = np.reshape(nn_params[((input_layer_size+1)*hidden_layer_size):], (num_labels, hidden_layer_size+1))
    
    m = X.shape[0]
    J = 0
    
    #recode y
    y_new = pd.get_dummies(y).as_matrix() # 5000*10
    
    #feedforwad
    a1, a2, a3, z2, z3 = predict(Theta1, Theta2, X, m)
    
    #cost
    J = 1/m*(-np.sum((y_new * np.log(a3)))-np.sum((1-y_new)*np.log(1-a3)))\
    + lam/(2*m) * (np.sum(np.square(Theta1[:,1:])) + np.sum(np.square(Theta2[:,1:])))
    
    return(J)

In [29]:
nnCostFunction(para, 400, 25, 10, X, y, 1)

0.38376985909092359

In [30]:
#gradient function
def nnGradientFun(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lam):
    Theta1 = np.reshape(nn_params[:((input_layer_size+1)*hidden_layer_size)], (hidden_layer_size, input_layer_size+1))
    Theta2 = np.reshape(nn_params[((input_layer_size+1)*hidden_layer_size):], (num_labels, hidden_layer_size+1))
    
    Theta1_grad = np.zeros(Theta1.size) # 25x401
    Theta2_grad = np.zeros(Theta2.size) # 10x26
    m = X.shape[0]
    #recode y
    y_new = pd.get_dummies(y).as_matrix() # 5000*10
    
    a1, a2, a3, z2, z3 = predict(Theta1, Theta2, X, m)
    #gradient
    d3 = a3 - y_new # 5000x10
    d2 = d3.dot(Theta2[:,1:])*sigmoidGradient(z2) # 5000x10*10x25 = 5000x25
    
    Theta2_grad = 1/m*d3.T.dot(a2) + lam/m*np.column_stack((np.zeros(num_labels), Theta2[:,1:]))# 10x5000*5000x26 
    Theta1_grad = 1/m*d2.T.dot(a1) + lam/m*np.column_stack((np.zeros(hidden_layer_size), Theta1[:, 1:])) # 25x5000*5000x401
    return(np.append(Theta2_grad.flatten(), Theta1_grad.flatten()))

#### Gradient checking

\begin{equation}
f_i(\theta) \approx \frac{J(\theta^{(i+)})-J(\theta^{(i-)})}{2\epsilon}
\end{equation}

In [31]:
def checkNNGradients(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lam, eps=10^(-4)):
    n = nn_params.size
    numGrad = np.zeros(n)
    
    paraPlus = np.tile(nn_params, (n, 1)) + np.diag(np.ones(n))*eps
    paraMinus = np.tile(nn_params, (n, 1)) - np.diag(np.ones(n))*eps
    
    for i in range(0, 3):
        costPlus = nnCostFunction(paraPlus[i,:], input_layer_size, hidden_layer_size, num_labels, X, y, lam)
        costMinus = nnCostFunction(paraMinus[i, :], input_layer_size, hidden_layer_size, num_labels, X, y, lam)
        numGrad[i] = (costPlus - costMinus)/2/eps
    return(numGrad)

In [32]:
checkNNGradients(para, 400, 25, 10, X, y, 1)

array([  1.80912511e-03,  -2.11248241e-12,   4.38829528e-13, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])

In [33]:
nnGradientFun(para, 400, 25, 10, X, y, 1)

array([  6.28737643e-04,   5.08457277e-04,   7.84221985e-05, ...,
         4.69418663e-09,   2.83929031e-09,   1.75890906e-12])

In [34]:
minimize(nnCostFunction, para, args=(400, 25, 10, X, y, 1), jac=nnGradientFun, options={'maxiter':50})

      fun: 0.38376985909092359
 hess_inv: array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])
      jac: array([  6.28737643e-04,   5.08457277e-04,   7.84221985e-05, ...,
         4.69418663e-09,   2.83929031e-09,   1.75890906e-12])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 26
      nit: 0
     njev: 14
   status: 2
  success: False
        x: array([ -2.25623899e-02,  -1.05624163e-08,   2.19414684e-09, ...,
        -2.47795788e-01,   1.28009118e+00,  -1.32752042e+00])